In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("preprocessedEmployee.csv", index_col=None)
df2 = dataset
df2 = pd.get_dummies(df2, drop_first=True, dtype=int)


In [3]:
dataset

,Age,Years_At_Company,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Promotions,Department,Gender,Job_Title,Education_Level,Resigned
0,55,2,5.0,6750,33,32,0,IT,Male,Specialist,High School,False
1,29,0,5.0,7500,34,34,2,Finance,Male,Developer,High School,False
2,55,8,3.0,5850,37,27,0,Finance,Male,Specialist,High School,False
3,48,7,2.0,4800,52,10,1,Customer Support,Female,Analyst,Bachelor,False
4,36,3,2.0,4800,38,11,1,Engineering,Female,Analyst,Bachelor,False
...,...,...,...,...,...,...,...,...,...,...,...,...
244,45,4,5.0,9000,48,9,1,HR,Female,Manager,Bachelor,False
245,32,2,4.0,6300,55,46,2,IT,Female,Specialist,Bachelor,False
246,24,7,4.0,4900,30,33,0,Marketing,Female,Technician,Bachelor,False
247,43,3,2.0,4800,57,36,1,HR,Female,Analyst,Bachelor,False


In [4]:
df2

,Age,Years_At_Company,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Promotions,Resigned,Department_Engineering,Department_Finance,...,Gender_Other,Job_Title_Consultant,Job_Title_Developer,Job_Title_Engineer,Job_Title_Manager,Job_Title_Specialist,Job_Title_Technician,Education_Level_High School,Education_Level_Master,Education_Level_PhD
0,55,2,5.0,6750,33,32,0,False,0,0,...,0,0,0,0,0,1,0,1,0,0
1,29,0,5.0,7500,34,34,2,False,0,1,...,0,0,1,0,0,0,0,1,0,0
2,55,8,3.0,5850,37,27,0,False,0,1,...,0,0,0,0,0,1,0,1,0,0
3,48,7,2.0,4800,52,10,1,False,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36,3,2.0,4800,38,11,1,False,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,45,4,5.0,9000,48,9,1,False,0,0,...,0,0,0,0,1,0,0,0,0,0
245,32,2,4.0,6300,55,46,2,False,0,0,...,0,0,0,0,0,1,0,0,0,0
246,24,7,4.0,4900,30,33,0,False,0,0,...,0,0,0,0,0,0,1,0,0,0
247,43,3,2.0,4800,57,36,1,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
dataset.columns

Index(['Age', 'Years_At_Company', 'Performance_Score', 'Monthly_Salary',
       'Work_Hours_Per_Week', 'Projects_Handled', 'Promotions', 'Department',
       'Gender', 'Job_Title', 'Education_Level', 'Resigned'],
      dtype='object')

In [6]:
indep_X=df2.drop('Resigned', axis=1)
dep_Y=df2['Resigned']

Feature Selection

Recursive Feature Elimination

In [11]:
def Linear(X_train, y_train, X_test):       
    from sklearn.linear_model import LogisticRegression  # Changed to LogisticRegression
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy

def Decision(X_train, y_train, X_test):
    from sklearn.tree import DecisionTreeClassifier  
    classifier = DecisionTreeClassifier(random_state=0)
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy

def random(X_train, y_train, X_test):       
    from sklearn.ensemble import RandomForestClassifier 
    classifier = RandomForestClassifier(n_estimators=10, random_state=0)
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy

def xgboost(X_train, y_train, X_test):       
    from xgboost import XGBClassifier 
    classifier = XGBClassifier(n_jobs=5, learning_rate=0.1, max_depth=10, random_state=1)
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy


In [17]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)

        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

In [19]:
def accuracy_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [21]:
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    colnames_list = []  
    accuracy_values = []  #  accuracy_values

    from sklearn.linear_model import LogisticRegression  #  LogisticRegression
    lin = LogisticRegression(random_state=0)

    from sklearn.tree import DecisionTreeClassifier  #  DecisionTreeClassifier
    dec = DecisionTreeClassifier(random_state=0)

    from sklearn.ensemble import RandomForestClassifier  #  RandomForestClassifier
    rf = RandomForestClassifier(n_estimators=10, random_state=0)

    from xgboost import XGBClassifier  # XGBClassifier
    xgb = XGBClassifier(n_jobs=5, learning_rate=0.1, max_depth=10, random_state=1)

    rfemodellist = [lin, dec, rf, xgb]

    for model in rfemodellist:
        log_rfe = RFE(estimator=model, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)

        # Get the column names selected by RFE
        selected_columns = [col for col, selected in zip(indep_X.columns, log_rfe.support_) if selected]
        colnames_list.append(selected_columns)

        # Fit the model and calculate accuracy
        X_train, X_test, y_train, y_test = split_scalar(pd.DataFrame(log_rfe_feature), dep_Y)
        model.fit(X_train, y_train)
        accuracy = accuracy_prediction(model, X_test, y_test) 
        accuracy_values.append(accuracy)

    return rfelist, colnames_list, accuracy_values

# Call the function with your data
rfelist, colnames_list, accuracy_values = rfeFeature(indep_X, dep_Y, 5)

# Print the selected column names and accuracy values for each model
for model_name, selected_columns, accuracy_value in zip(["Linear", "Decision", "Random", "XGBoost"], colnames_list, accuracy_values):
    print(f"Model: {model_name}")
    print("Selected Columns:", selected_columns)
    print(f"Accuracy Value: {accuracy_value}\n")  # Changed from R2 Value to Accuracy Value



C:\Users\sindhiya maria\Python_project\Citation_20_page\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\sindhiya maria\Python_project\Citation_20_page\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Model: Linear
Selected Columns: ['Department_Finance', 'Department_IT', 'Department_Legal', 'Education_Level_High School', 'Education_Level_Master']
Accuracy Value: 0.9047619047619048

Model: Decision
Selected Columns: ['Age', 'Years_At_Company', 'Monthly_Salary', 'Work_Hours_Per_Week', 'Projects_Handled']
Accuracy Value: 0.7619047619047619

Model: Random
Selected Columns: ['Age', 'Years_At_Company', 'Monthly_Salary', 'Work_Hours_Per_Week', 'Projects_Handled']
Accuracy Value: 0.873015873015873

Model: XGBoost
Selected Columns: ['Monthly_Salary', 'Promotions', 'Department_Legal', 'Job_Title_Specialist', 'Education_Level_High School']
Accuracy Value: 0.9047619047619048



In [ ]:
pip install xgboost

Model Creation

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np

# Example: Assume you already have indep_X, dep_Y from preprocessing
X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)

# Wrap DecisionTree with RFE
base_clf = DecisionTreeClassifier(random_state=0)

# Select top N features using RFE
selector = RFE(estimator=base_clf, n_features_to_select=5)  # adjust N as needed
X_train_rfe = selector.fit_transform(X_train, y_train)
X_test_rfe = selector.transform(X_test)

# Define parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_features': [None, 'sqrt', 'log2']
}

# Run GridSearchCV on the RFE-transformed data
grid = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=0),
    param_grid=param_grid,
    refit=True,
    verbose=3,
    n_jobs=-1,
    scoring='f1_weighted'
)

grid.fit(X_train_rfe, y_train)

# Predictions
y_predict = grid.predict(X_test_rfe)

# Evaluation
cm = confusion_matrix(y_test, y_predict)
clf_report = classification_report(y_test, y_predict)
f1_macro = f1_score(y_test, y_predict, average='weighted')

print("Best Parameters:", grid.best_params_)
print("F1 Weighted Score:", f1_macro)
print("\nClassification Report:\n", clf_report)
print("\nConfusion Matrix:\n", cm)

# Optional: See which features RFE kept
selected_features = np.array(indep_X.columns)[selector.support_]
print("\nSelected Features by RFE:", selected_features)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'random'}
F1 Weighted Score: 0.8620052513857823

Classification Report:
               precision    recall  f1-score   support

       False       0.93      0.91      0.92        57
        True       0.29      0.33      0.31         6

    accuracy                           0.86        63
   macro avg       0.61      0.62      0.61        63
weighted avg       0.87      0.86      0.86        63


Confusion Matrix:
 [[52  5]
 [ 4  2]]

Selected Features by RFE: ['Age' 'Years_At_Company' 'Performance_Score' 'Work_Hours_Per_Week'
 'Projects_Handled']


In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)

# Apply RFE
base_clf = DecisionTreeClassifier(random_state=0)
selector = RFE(estimator=base_clf, n_features_to_select=5)
X_train_rfe = selector.fit_transform(X_train, y_train)
X_test_rfe = selector.transform(X_test)  # Transform test data with same RFE

# Parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_features': [None, 'sqrt', 'log2']
}

# GridSearchCV
grid = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=0),
    param_grid=param_grid,
    refit=True,
    verbose=3,
    n_jobs=-1,
    scoring='f1_weighted'
)

grid.fit(X_train_rfe, y_train)

# Predictions using RFE-transformed data
y_predict = grid.predict(X_test_rfe)
y_proba = grid.predict_proba(X_test_rfe)[:, 1]  # Probabilities for positive class

# Evaluation metrics
cm = confusion_matrix(y_test, y_predict)
clf_report = classification_report(y_test, y_predict)
f1_weighted = f1_score(y_test, y_predict, average='weighted')

# ✅ CORRECT ROC AUC calculation
roc_auc = roc_auc_score(y_test, y_proba)

# Print results
print("Best Parameters:", grid.best_params_)
print(f"F1 Weighted Score: {f1_weighted:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

print("\nClassification Report:")
print(clf_report)

print("Confusion Matrix:")
print(cm)

# Selected features
selected_features = np.array(indep_X.columns)[selector.support_]
print("Selected Features by RFE:", selected_features)

# Additional performance metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

accuracy = accuracy_score(y_test, y_predict)
precision_pos = precision_score(y_test, y_predict, pos_label=True)
recall_pos = recall_score(y_test, y_predict, pos_label=True)

print(f"\nDetailed Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Precision (Resigned): {precision_pos:.4f}")
print(f"Recall (Resigned): {recall_pos:.4f}")
print(f"F1 Weighted: {f1_weighted:.4f}")


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'random'}
F1 Weighted Score: 0.8620
ROC AUC Score: 0.6228

Classification Report:
              precision    recall  f1-score   support

       False       0.93      0.91      0.92        57
        True       0.29      0.33      0.31         6

    accuracy                           0.86        63
   macro avg       0.61      0.62      0.61        63
weighted avg       0.87      0.86      0.86        63

Confusion Matrix:
[[52  5]
 [ 4  2]]
Selected Features by RFE: ['Age' 'Years_At_Company' 'Performance_Score' 'Work_Hours_Per_Week'
 'Projects_Handled']

Detailed Performance Metrics:
Accuracy: 0.8571
ROC AUC: 0.6228
Precision (Resigned): 0.2857
Recall (Resigned): 0.3333
F1 Weighted: 0.8620


In [27]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test_rfe)[:,1])

0.6228070175438596

In [29]:
table=pd.DataFrame.from_dict(grid.cv_results_)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006941,0.003716,0.008852,0.000747,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.719363,0.820025,0.765883,0.729696,0.644984,0.735990,0.057539,8
1,0.004879,0.002156,0.007301,0.002345,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.714514,0.713857,0.774506,0.783784,0.729730,0.743278,0.029975,6
2,0.003943,0.002222,0.005994,0.003501,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.754214,0.729730,0.759331,0.711197,0.663063,0.723507,0.034843,11
3,0.003843,0.001769,0.009165,0.003065,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.714514,0.745114,0.848736,0.801756,0.837838,0.789591,0.052132,1
4,0.004433,0.001665,0.008224,0.001619,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.754214,0.729730,0.759331,0.711197,0.663063,0.723507,0.034843,11
5,0.005071,0.001325,0.009219,0.002050,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.714514,0.745114,0.848736,0.801756,0.837838,0.789591,0.052132,1
6,0.007404,0.001694,0.008812,0.001846,entropy,None,best,"{'criterion': 'entropy', 'max_features': None,...",0.789212,0.729730,0.838933,0.729730,0.644984,0.746518,0.065171,5
7,0.005391,0.000698,0.010448,0.000254,entropy,None,random,"{'criterion': 'entropy', 'max_features': None,...",0.699595,0.783784,0.774506,0.765883,0.680549,0.740863,0.042287,7
8,0.004982,0.002233,0.008721,0.001809,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.736842,0.713857,0.765883,0.713857,0.713857,0.728859,0.020541,9
9,0.004980,0.001079,0.008416,0.003148,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.714514,0.745114,0.869935,0.801756,0.801756,0.786615,0.053519,3


In [31]:
# Save the best model
filename = "Decision_tree_employee_final.sav"
pickle.dump(grid, open(filename, 'wb'))

In [33]:
# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))

Employee Input Data 

In [40]:
def get_categorical_prediction(prediction):
    return "Yes" if prediction == 1 else "No"

# Collect employee-related inputs
Age_input = float(input("Age: "))
Years_At_Company_input = float(input("Years at Company: "))
Performance_Score_input = float(input("Performance Score: "))
#Monthly_Salary_input = float(input("Monthly Salary: "))
Work_Hours_Per_Week_input = float(input("Work Hours Per Week: "))
Projects_Handled_input = float(input("Projects Handled: "))
#Promotions_input = float(input("Promotions: "))


future_prediction = loaded_model.predict([[
    Age_input,
    Years_At_Company_input,
    Performance_Score_input,
    Work_Hours_Per_Week_input,
    Projects_Handled_input,
   
]])


# Predict the future maintenance requirement
#future_prediction = loaded_model.predict([[Age,Years at Company,Performance Score,Monthly Salary,Work Hours Per Week,Projects Handled,Promotions]])


# Convert numerical prediction to categorical label
future_prediction_categorical = get_categorical_prediction(future_prediction[0])

Age:  32
Years at Company:  2
Performance Score:  3.0
Work Hours Per Week:  35
Projects Handled:  43


In [46]:
# Convert numerical prediction to categorical label
future_prediction_categorical = get_categorical_prediction(future_prediction[0])


# Print the categorical prediction
print("Future Prediction (Resigned):", future_prediction_categorical)

Future Prediction (Resigned): No


In [44]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# 1. Save the complete model pipeline
print("="*60)
print("SAVING MODEL FOR DEPLOYMENT")
print("="*60)

# Create a complete model package
model_package = {
    'trained_model': grid,                    # GridSearchCV object with best model
    'rfe_selector': selector,                 # RFE selector fitted on training data
    'selected_features': selected_features,   # Names of selected features
    'feature_columns': list(indep_X.columns), # Original column names
    'best_params': grid.best_params_,
    'cv_score': grid.best_score_,
    'model_type': 'DecisionTree_RFE'
}

filename = "Decision_tree_employee_final.sav"
pickle.dump(model_package, open(filename, 'wb'))
print(f"Model saved as: {filename}")
print(f"Best Parameters: {grid.best_params_}")
print(f"CV Score: {grid.best_score_:.4f}")

# 2. Test loading and using the saved model
print("\n" + "="*60)
print("TESTING SAVED MODEL")
print("="*60)

# Load the saved model
loaded_model_package = pickle.load(open(filename, 'rb'))

print("Model loaded successfully!")
print(f"Model type: {loaded_model_package['model_type']}")
print(f"Selected features: {loaded_model_package['selected_features']}")

# 3. Test with original test data
print("\n" + "="*60)
print("TESTING ON ORIGINAL TEST SET")
print("="*60)

# Extract components from loaded model
loaded_grid = loaded_model_package['trained_model']
loaded_selector = loaded_model_package['rfe_selector']

# Apply RFE transformation to test data (same as during training)
X_test_rfe_loaded = loaded_selector.transform(X_test)

# Make predictions using loaded model
y_pred_loaded = loaded_grid.predict(X_test_rfe_loaded)
y_proba_loaded = loaded_grid.predict_proba(X_test_rfe_loaded)[:, 1]

# Calculate metrics
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
roc_auc_loaded = roc_auc_score(y_test, y_proba_loaded)
cm_loaded = confusion_matrix(y_test, y_pred_loaded)

print(f"Loaded Model Performance:")
print(f"Accuracy: {accuracy_loaded:.4f}")
print(f"ROC AUC: {roc_auc_loaded:.4f}")
print(f"\nConfusion Matrix:")
print(cm_loaded)
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_loaded))

# 4. Create a prediction function for new data
print("\n" + "="*60)
print("CREATING PREDICTION FUNCTION")
print("="*60)

def predict_employee_resignation(employee_data, model_file="Decision_tree_employee_final.sav"):
    """
    Predict employee resignation probability
    
    Parameters:
    employee_data: pandas DataFrame with employee features (same columns as training)
    model_file: path to saved model file
    
    Returns:
    predictions: array of predictions (0=No Resignation, 1=Resignation)
    probabilities: array of resignation probabilities
    """
    
    # Load the model
    model_package = pickle.load(open(model_file, 'rb'))
    trained_model = model_package['trained_model']
    rfe_selector = model_package['rfe_selector']
    
    # Apply same RFE transformation
    employee_data_rfe = rfe_selector.transform(employee_data)
    
    # Make predictions
    predictions = trained_model.predict(employee_data_rfe)
    probabilities = trained_model.predict_proba(employee_data_rfe)[:, 1]
    
    return predictions, probabilities

# Test the prediction function
print("Testing prediction function...")

# 5. Test with new sample data
print("\n" + "="*60)
print("TESTING WITH SAMPLE EMPLOYEE DATA")
print("="*60)

# Create sample employee data (using first 3 rows of test set as example)
sample_employees = X_test.iloc[:3].copy()
actual_outcomes = y_test.iloc[:3].values

print("Sample Employee Data:")
print(sample_employees)

# Make predictions using the function
sample_predictions, sample_probabilities = predict_employee_resignation(sample_employees)

print(f"\nPredictions Results:")
for i in range(len(sample_predictions)):
    prediction_text = "Will Resign" if sample_predictions[i] == 1 else "Will Not Resign"
    print(f"Employee {i+1}: {prediction_text} (Probability: {sample_probabilities[i]:.4f})")
    print(f"  Actual outcome: {'Resigned' if actual_outcomes[i] == 1 else 'Did not resign'}")

# 6. Test model consistency
print("\n" + "="*60)
print("VERIFYING MODEL CONSISTENCY")
print("="*60)

# Compare original predictions with loaded model predictions
consistency_check = np.array_equal(y_pred_loaded, y_predict)
probability_difference = np.abs(y_proba_loaded - y_proba).mean()

print(f"Predictions match original model: {consistency_check}")
print(f"Average probability difference: {probability_difference:.8f}")

if consistency_check and probability_difference < 1e-10:
    print(" Model saved and loaded successfully!")
    print(" Predictions are consistent!")
else:
    print("❌ Warning: Model predictions are inconsistent!")

print("\n" + "="*60)
print("MODEL READY FOR DEPLOYMENT")
print("="*60)
print(f"Use predict_employee_resignation() function with new employee data")
print(f"Model file: {filename}")
print(f"Selected features: {loaded_model_package['selected_features']}")


SAVING MODEL FOR DEPLOYMENT
Model saved as: Decision_tree_employee_final.sav
Best Parameters: {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'random'}
CV Score: 0.7896

TESTING SAVED MODEL
Model loaded successfully!
Model type: DecisionTree_RFE
Selected features: ['Age' 'Years_At_Company' 'Performance_Score' 'Work_Hours_Per_Week'
 'Projects_Handled']

TESTING ON ORIGINAL TEST SET
Loaded Model Performance:
Accuracy: 0.8571
ROC AUC: 0.6228

Confusion Matrix:
[[52  5]
 [ 4  2]]

Classification Report:
              precision    recall  f1-score   support

       False       0.93      0.91      0.92        57
        True       0.29      0.33      0.31         6

    accuracy                           0.86        63
   macro avg       0.61      0.62      0.61        63
weighted avg       0.87      0.86      0.86        63


CREATING PREDICTION FUNCTION
Testing prediction function...

TESTING WITH SAMPLE EMPLOYEE DATA
Sample Employee Data:
     Age  Years_At_Company  Performance_